In [79]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importamos bibliotecas necesarias

In [106]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tabulate import tabulate
from sklearn.utils import resample, shuffle

Leemos el csv y eliminamos la columna de Customer_Number ya que no la necesitamos para el modelo

In [107]:
df = pd.read_csv('/content/drive/MyDrive/Basic Data Analytics/6.-Project/Credit classification/creditcardmarketing.csv' ,sep=';')
df=df.drop(columns=['Customer_Number'])
df

,Offer_Accepted,Reward,Mailer_Type,Income_Level,Bank_Accounts_Open,Overdraft_Protection,Credit_Rating,Credit_Cards_Held,Homes_Owned,Household_Size,Own_Your_Home,Average_Balance,Q1_Balance,Q2_Balance,Q3_Balance,Q4_Balance
0,No,Air Miles,Letter,High,1,No,High,2,1,4,No,1160.75,1669.0,877.0,1095.0,1002.0
1,No,Air Miles,Letter,Medium,1,No,Medium,2,2,5,Yes,147.25,39.0,106.0,78.0,366.0
2,No,Air Miles,Postcard,High,2,No,Medium,2,1,2,Yes,276.50,367.0,352.0,145.0,242.0
3,No,Air Miles,Letter,Medium,2,No,High,1,1,4,No,1219.00,1578.0,1760.0,1119.0,419.0
4,No,Air Miles,Letter,Medium,1,No,Medium,2,1,6,Yes,1211.00,2140.0,1357.0,982.0,365.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,No,Cash Back,Letter,High,1,No,Low,1,1,5,Yes,167.50,136.0,65.0,71.0,398.0
17996,No,Cash Back,Letter,High,1,No,Low,3,1,3,Yes,850.50,984.0,940.0,943.0,535.0
17997,No,Cash Back,Letter,High,1,No,Low,2,1,4,No,1087.25,918.0,767.0,1170.0,1494.0
17998,No,Cash Back,Letter,Medium,1,No,Medium,4,2,2,Yes,1022.25,626.0,983.0,865.0,1615.0


**ANÁLISIS DESCRIPTIVO**

Columnas

In [108]:
df.columns

Index(['Offer_Accepted', 'Reward', 'Mailer_Type', 'Income_Level',
       'Bank_Accounts_Open', 'Overdraft_Protection', 'Credit_Rating',
       'Credit_Cards_Held', 'Homes_Owned', 'Household_Size', 'Own_Your_Home',
       'Average_Balance', 'Q1_Balance', 'Q2_Balance', 'Q3_Balance',
       'Q4_Balance'],
      dtype='object')

Estadísticas descriptivas (numero de datos, media, desviación, valor mínimo, valor máximo, Q0.25, Q0.5 y Q0.75 de las columnas numéricas)

In [109]:
df.describe()

,Bank_Accounts_Open,Credit_Cards_Held,Homes_Owned,Household_Size,Average_Balance,Q1_Balance,Q2_Balance,Q3_Balance,Q4_Balance
count,18000.000000,18000.000000,18000.000000,18000.000000,17976.000000,17976.000000,17976.000000,17976.000000,17976.000000
mean,1.255778,1.903500,1.203444,3.499056,940.515562,910.450656,999.392190,1042.033600,810.185803
std,0.472501,0.797009,0.427341,1.114182,350.297837,620.077060,457.402268,553.452599,559.001365
min,1.000000,1.000000,1.000000,1.000000,48.250000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,3.000000,787.500000,392.750000,663.000000,633.000000,363.000000
50%,1.000000,2.000000,1.000000,3.000000,1007.000000,772.000000,1032.000000,945.500000,703.000000
75%,1.000000,2.000000,1.000000,4.000000,1153.250000,1521.000000,1342.000000,1463.000000,1212.000000
max,3.000000,4.000000,3.000000,9.000000,3366.250000,3450.000000,3421.000000,3823.000000,4215.000000


Información (Tipos de datos y cantidad de datos no nulos por columna)

In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Offer_Accepted        18000 non-null  object 
 1   Reward                18000 non-null  object 
 2   Mailer_Type           18000 non-null  object 
 3   Income_Level          18000 non-null  object 
 4   Bank_Accounts_Open    18000 non-null  int64  
 5   Overdraft_Protection  18000 non-null  object 
 6   Credit_Rating         18000 non-null  object 
 7   Credit_Cards_Held     18000 non-null  int64  
 8   Homes_Owned           18000 non-null  int64  
 9   Household_Size        18000 non-null  int64  
 10  Own_Your_Home         18000 non-null  object 
 11  Average_Balance       17976 non-null  float64
 12  Q1_Balance            17976 non-null  float64
 13  Q2_Balance            17976 non-null  float64
 14  Q3_Balance            17976 non-null  float64
 15  Q4_Balance         

Comprobar que efectivamente Household_Size, Credit_Cards_Held y Homes_Owned son columnas categoricas(por si acaso)

In [111]:
df['Household_Size'].value_counts(),df['Credit_Cards_Held'].value_counts(),df['Homes_Owned'].value_counts()

(3    5712
 4    5543
 5    2853
 2    2813
 1     539
 6     538
 8       1
 9       1
 Name: Household_Size, dtype: int64,
 2    7944
 1    6154
 3    3387
 4     515
 Name: Credit_Cards_Held, dtype: int64,
 1    14523
 2     3292
 3      185
 Name: Homes_Owned, dtype: int64)

**PREPARACION DEL CSV**

Ver la cantidad de datos que hay de cada clase en la columna Offer_Accepted, la cual será nuestra futura 'y' en nuestro modelo de clasificacion y queremos que la cantidad de datos de cada clase sea parecida para obtener una buena predicion.

In [112]:
df['Offer_Accepted'].value_counts()

No     16977
Yes     1023
Name: Offer_Accepted, dtype: int64

Hemos visto que la diferencia de cantidad de datos de cada clase es muy grande por tanto vamos a balancear las clases con resample  

In [113]:
# Suponiendo que tu DataFrame es df y la columna objetivo es 'Clasificacion'
clase_mayoritaria = df[df['Offer_Accepted'] == 'No']
clase_minoritaria = df[df['Offer_Accepted'] == 'Yes']

# Undersampling (eliminar aleatoriamente ejemplos de la clase mayoritaria)
clase_mayoritaria_undersampled = resample(clase_mayoritaria, replace=False, n_samples=len(clase_minoritaria), random_state=42)

# Concatenar la clase minoritaria con la clase mayoritaria undersampled
df = pd.concat([clase_mayoritaria_undersampled, clase_minoritaria])

# Mostrar el nuevo DataFrame balanceado
df

,Offer_Accepted,Reward,Mailer_Type,Income_Level,Bank_Accounts_Open,Overdraft_Protection,Credit_Rating,Credit_Cards_Held,Homes_Owned,Household_Size,Own_Your_Home,Average_Balance,Q1_Balance,Q2_Balance,Q3_Balance,Q4_Balance
10135,No,Cash Back,Postcard,Medium,1,Yes,Medium,1,1,3,Yes,741.25,797.0,903.0,729.0,536.0
5937,No,Cash Back,Letter,High,1,No,Low,1,1,4,Yes,967.00,1999.0,1134.0,644.0,91.0
12829,No,Air Miles,Postcard,Low,2,No,Low,1,1,1,Yes,652.50,776.0,679.0,504.0,651.0
7057,No,Points,Postcard,Low,2,No,Medium,1,1,4,Yes,1199.00,1033.0,1800.0,923.0,1040.0
4109,No,Air Miles,Postcard,Low,1,No,High,2,2,3,No,1515.75,1454.0,1318.0,2275.0,1016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17940,Yes,Air Miles,Postcard,Medium,1,Yes,Low,2,1,4,Yes,1140.25,634.0,1076.0,1966.0,885.0
17942,Yes,Air Miles,Postcard,Low,1,No,Medium,1,1,3,Yes,1026.25,297.0,1672.0,1569.0,567.0
17971,Yes,Cash Back,Postcard,Low,1,No,High,2,2,4,Yes,1221.75,2118.0,1632.0,1029.0,108.0
17984,Yes,Points,Postcard,High,1,No,Low,2,1,2,No,977.25,1628.0,1025.0,885.0,371.0


Mezclar las filas del dtf ya que primero aparecen todo los noes y despues los sies y queremos que se intercalen para un mejor aprendizaje

In [114]:
df = shuffle(df, random_state=42)
df = df.reset_index(drop=True)
df

,Offer_Accepted,Reward,Mailer_Type,Income_Level,Bank_Accounts_Open,Overdraft_Protection,Credit_Rating,Credit_Cards_Held,Homes_Owned,Household_Size,Own_Your_Home,Average_Balance,Q1_Balance,Q2_Balance,Q3_Balance,Q4_Balance
0,No,Points,Postcard,Low,2,No,High,1,1,4,Yes,761.00,729.0,502.0,866.0,947.0
1,Yes,Air Miles,Postcard,High,1,No,Low,1,1,4,No,1647.25,1780.0,1648.0,1659.0,1502.0
2,No,Points,Postcard,Medium,1,No,High,1,1,6,Yes,1199.75,1372.0,1687.0,908.0,832.0
3,No,Cash Back,Postcard,Medium,1,Yes,Low,4,2,4,Yes,876.00,246.0,770.0,950.0,1538.0
4,Yes,Air Miles,Postcard,High,1,No,Low,3,1,5,Yes,666.50,684.0,807.0,569.0,606.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2041,Yes,Cash Back,Postcard,Medium,1,No,Medium,2,1,3,No,1609.75,1455.0,1265.0,1834.0,1885.0
2042,Yes,Cash Back,Postcard,Low,2,No,Low,1,1,4,No,1066.75,1657.0,1435.0,913.0,262.0
2043,No,Air Miles,Letter,High,2,No,Low,1,1,4,Yes,308.25,124.0,458.0,202.0,449.0
2044,Yes,Points,Postcard,Medium,1,No,Medium,2,1,3,Yes,1152.00,472.0,1462.0,1357.0,1317.0


Aplicar codificación one-hot a las columnas categóricas y estandarizar las columnas numéricas de nuestra futura 'X'. Para ello:\
1.Definir dos listas, categoricas y numericas, que contienen los nombres de las columnas que se consideran categóricas y numéricas, respectivamente.\
2.Crear dos pipelines de transformación utilizando la clase Pipeline de scikit-learn:\
  -Para las columnas categóricas (cat_pipe):
    SimpleImputer: Se utiliza para manejar valores faltantes en las columnas categóricas, llenando los valores faltantes con la cadena 'missing'.
    OneHotEncoder: Se utiliza para realizar codificación one-hot en las columnas categóricas.\
  -Para las columnas numéricas (num_pipe):
    SimpleImputer: Se utiliza para manejar valores faltantes en las columnas numéricas, llenando los valores faltantes con la mediana.
    StandardScaler: Se utiliza para estandarizar las columnas numéricas (restar la media y escalar a la varianza unitaria).\
3.Utilizar la clase ColumnTransformer para aplicar a columnas categoricas y numericas sus respectivos pipelines.


In [115]:
categoricas = ['Reward',	'Mailer_Type',	'Income_Level',	'Bank_Accounts_Open',	'Overdraft_Protection',	'Credit_Rating',	'Credit_Cards_Held',	'Homes_Owned',	'Household_Size',	'Own_Your_Home']
print(f"Las columnas categoricas son: {categoricas}")

#Define numeric columns
numericas = ['Average_Balance',	'Q1_Balance',	'Q2_Balance',	'Q3_Balance',	'Q4_Balance']
print(f"Las columnas numericas son: {numericas}")


cat_pipe = Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))])


num_pipe = Pipeline([('imputer', SimpleImputer(strategy='median')),
                     ('scaler', StandardScaler())])


preprocessor = ColumnTransformer(transformers=[('cat', cat_pipe, categoricas),
                                               ('num', num_pipe, numericas)])

preprocessor.fit(df)

x = pd.DataFrame(preprocessor.transform(df))

display(x)

Las columnas categoricas son: ['Reward', 'Mailer_Type', 'Income_Level', 'Bank_Accounts_Open', 'Overdraft_Protection', 'Credit_Rating', 'Credit_Cards_Held', 'Homes_Owned', 'Household_Size', 'Own_Your_Home']
Las columnas numericas son: ['Average_Balance', 'Q1_Balance', 'Q2_Balance', 'Q3_Balance', 'Q4_Balance']


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,-0.546071,-0.274453,-1.123254,-0.351139,0.237790
1,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,2.030157,1.426826,1.374697,1.067161,1.246297
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.729325,0.766387,1.459706,-0.276021,0.028820
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,-0.211779,-1.056297,-0.539091,-0.200903,1.311713
4,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,-0.820771,-0.347296,-0.458442,-0.882331,-0.381851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2041,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.921148,0.900741,0.539867,1.380152,1.942257
2042,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.342710,1.227723,0.910418,-0.267078,-1.006944
2043,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,-1.862163,-1.253781,-1.219162,-1.538719,-0.667141
2044,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.590522,-0.690465,0.969270,0.527026,0.910128


Mapear la columna Offer_Accepted (nuestra futura 'y') para que los datos sean de tipo numerico y no objeto

In [116]:
df['Offer_Accepted']=df['Offer_Accepted'].map({'No': 0, 'Yes': 1})
df['Offer_Accepted']

0       0
1       1
2       0
3       0
4       1
       ..
2041    1
2042    1
2043    0
2044    1
2045    1
Name: Offer_Accepted, Length: 2046, dtype: int64

**CREACIÓN DEL MODELO**

Dividir los datos en train y test con train_test_split

In [117]:
X_train, X_test, y_train, y_test = train_test_split(x, df['Offer_Accepted'], train_size = 0.8, random_state = 0)

Entrenar y evaluar modelo random forest

In [118]:
# Entrenar un modelo Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Evaluar el modelo Random Forest
y_pred_rf = rf_model.predict(X_test)
print("Random Forest:")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:")
print(classification_report(y_test, y_pred_rf))

conf_matrix_logistic = confusion_matrix(y_test, y_pred_rf)
print(tabulate(conf_matrix_logistic,headers = ['pred No','pred Yes'], showindex = ['real No','real Yes'], tablefmt = 'fancy_grid'))

Random Forest:
Accuracy: 0.7146341463414634
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.70      0.72       215
           1       0.69      0.73      0.71       195

    accuracy                           0.71       410
   macro avg       0.71      0.72      0.71       410
weighted avg       0.72      0.71      0.71       410

╒══════════╤═══════════╤════════════╕
│          │   pred No │   pred Yes │
╞══════════╪═══════════╪════════════╡
│ real No  │       151 │         64 │
├──────────┼───────────┼────────────┤
│ real Yes │        53 │        142 │
╘══════════╧═══════════╧════════════╛
